# Hipóteses trabalhadas para identificar o modelo de dados

  1 - Separar em objetos diferentes normalizando os dados, ex: ano do evento, studios, produtores
    - Eleva a complexidade da solução tendo que trabalhar com joins
  2 - Alterar a granularidade dos objetos, como: studios e produtores
    - Mais simples permite explorar subqueries no modelo OBT (One-Big-Table)

Como os requisitos são específicos com a perguntas focadas em recursão a segunda hipótese é a mais interessante.

In [45]:
%uv add polars


/home/adriano/Arquivos/Projetos/app-be-os-piorfilme/analise/.venv/bin/python3: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import polars as pl


In [ ]:
# csv Movielist.csv schema
# year;title;studios;producers;winner

schema = pl.Schema(
    {
        "year": pl.Int32(),
        "title": pl.String(),
        "studios": pl.String(),
        "producers": pl.String(),
        "winner": pl.String(),
    }
)

df: pl.DataFrame = pl.read_csv(
    "Movielist.csv", schema=schema, separator=";", has_header=True, null_values=[""]
)


In [ ]:
df.head()


year,title,studios,producers,winner
i32,str,str,str,str
1980,"""Can't Stop the Music""","""Associated Film Distribution""","""Allan Carr""","""yes"""
1980,"""Cruising""","""Lorimar Productions, United Ar…","""Jerry Weintraub""",null
1980,"""The Formula""","""MGM, United Artists""","""Steve Shagan""",null
1980,"""Friday the 13th""","""Paramount Pictures""","""Sean S. Cunningham""",null
1980,"""The Nude Bomb""","""Universal Studios""","""Jennings Lang""",null


In [ ]:
# Função para dividir produtores por ',' e 'and'
import re


def split_producers(producers):
    if producers is None:
        return []
    return [p.strip() for p in re.split(r",| and ", producers) if p.strip()]


In [ ]:
# Aplica a função e explode as linhas
df = df.with_columns(
    pl.col("producers").map_elements(split_producers).alias("producers")
).explode("producers")

# Salva o resultado
df.write_csv("Movielist_Producers_one_per_line.csv", separator=";")


/tmp/ipykernel_26561/2074363542.py:2: MapWithoutReturnDtypeWarning: 'return_dtype' of function python_udf must be set

A later expression might fail because the output type is not known. Set return_dtype=pl.self_dtype() if the type is unchanged, or set the proper output data type.
  df = df.with_columns(


In [ ]:
# Etapas da construção da consulta

  1 - Separação dos produtores sendo um por linha
  2 - Contando vitórias
  3 - Identificando a primeira vitória
  4 - Identificando a próxima vitória através da remoção da primeira vitória
  5 - Calculando o intervalo entre vitórias


```console
select 
	films.producers,
	(select count(*) from films as f_s where f_s.producers = films.producers and f_s.winner = "yes") as wins,
	(
		(select min(year) from films as f_s_nw where f_s_nw.producers = films.producers and f_s_nw.winner = "yes" and f_s_nw.year not in (select min(year) from films as f_s_fw where f_s_fw.producers = films.producers and f_s_fw.winner = "yes")) -
		(select min(year) from films as f_s_fw where f_s_fw.producers = films.producers and f_s_fw.winner = "yes")
	) as wins_interval,
	(select min(year) from films as f_s_fw where f_s_fw.producers = films.producers and f_s_fw.winner = "yes") as first_win,
	
	(select min(year) from films as f_s_nw where f_s_nw.producers = films.producers and f_s_nw.winner = "yes" and f_s_nw.year not in (select min(year) from films as f_s_fw where f_s_fw.producers = films.producers and f_s_fw.winner = "yes")) as next_win
	
from films
GROUP by films.producers
ORDER by wins DESC
```
